In [12]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

#loading the data
x = np.load("results/x.npy")
y = np.load("results/y.npy")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y, random_state = 42)
print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

#Building up a 2D CNN model
model = models.Sequential([
    layers.Input(shape=(257, 31, 1)),

    layers.Conv2D(32, (7, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 1)),

    layers.Conv2D(64, (5, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 1)),

    layers.Conv2D(128, (3, 2), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.GlobalAveragePooling2D(),

    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])



model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics = ['accuracy'])

model.summary()

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, 
                              restore_best_weights = True)

history = model.fit(
    x_train, y_train,
    epochs = 30,
    batch_size = 32,
    validation_data = (x_test, y_test),
    callbacks = early_stopping
) 

x_train shape:  (750, 257, 31, 1)
x_test shape:  (250, 257, 31, 1)
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 257, 31, 32)       704       
                                                                 
 batch_normalization_21 (Ba  (None, 257, 31, 32)       128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 128, 31, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 128, 31, 64)       30784     
                                                                 
 batch_normalization_22 (Ba  (None, 128, 31, 64)       256       
 tchNormalization)                                   

In [13]:
y_prob = model.predict(x_test)

8/8 [==============================] - 1s 82ms/step


In [14]:
print("Prediction range:", np.min(y_prob), "to", np.max(y_prob))

Prediction range: 0.3752936 to 0.39474684


In [29]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# Load preprocessed spectrogram data
x = np.load("results/x.npy")
y = np.load("results/y.npy")

# Split into training and test sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, stratify=y, random_state=42)

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

# Define CNN model
model = models.Sequential([
    layers.Input(shape=(129, 30, 1)),

    layers.Conv2D(32, (7, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 1)),

    layers.Conv2D(64, (5, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 1)),

    layers.Conv2D(128, (3, 2), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.GlobalAveragePooling2D(),

    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# Setup early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)

# Evaluate predictions
y_prob = model.predict(x_test)
print("Prediction range:", np.min(y_prob), "to", np.max(y_prob))


x_train shape: (750, 129, 30, 1)
x_test shape: (250, 129, 30, 1)
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_66 (Conv2D)          (None, 129, 30, 32)       704       
                                                                 
 batch_normalization_72 (Ba  (None, 129, 30, 32)       128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_44 (MaxPooli  (None, 64, 30, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_67 (Conv2D)          (None, 64, 30, 64)        30784     
                                                                 
 batch_normalization_73 (Ba  (None, 64, 30, 64)        256       
 tchNormalization)                                    

After getting small improvements

In [25]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Load preprocessed spectrogram data
x = np.load("results/x.npy")
y = np.load("results/y.npy")

# Split into training and test sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, stratify=y, random_state=42)

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

# Define CNN model
model = tf.keras.Sequential([
    layers.Input(shape=(129, 30, 1)),
    layers.BatchNormalization(),

    layers.Conv2D(32, (5, 5), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.GlobalAveragePooling2D(),

    layers.Dense(64, activation='relu', kernel_regularizer=l2(1e-4)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05),
    metrics=['accuracy']
)

model.summary()

# Setup early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)

# Evaluate predictions
y_prob = model.predict(x_test)
print("Prediction range:", np.min(y_prob), "to", np.max(y_prob))


x_train shape: (750, 129, 30, 1)
x_test shape: (250, 129, 30, 1)
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_59 (Ba  (None, 129, 30, 1)        4         
 tchNormalization)                                               
                                                                 
 conv2d_54 (Conv2D)          (None, 129, 30, 32)       832       
                                                                 
 batch_normalization_60 (Ba  (None, 129, 30, 32)       128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_36 (MaxPooli  (None, 64, 15, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_55 (Conv2D)          (None, 64, 15, 64)       